# 2장. Data Collection
만약 충돌 회피 예제를 실행했다면, 다음 세 단계에 익숙할 것입니다.  
- 데이터 수집 Data collection
- 훈련 Train Model
- 배포 Live Demo

이 챕터에서는 똑같은 작업을 수행할 것입니다.  
하지만, 분류 대신에 회귀(Regression)라는 다른 기본 기술을 배우게 될 것입니다.  
이 회귀 기술은 인공지능 무인운반차량(AGV)이 도로(또는 실제로는 어떤 경로나 목표 지점이든)를 따라가도록 할 것입니다.  
경로상의 다양한 위치에 인공지능 무인운반차량(AGV)을 배치합니다. (중심으로부터의 오프셋, 다른 각도 등)
인공지능 무인운반차량(AGV)의 라이브 카메라 화면을 출력합니다.  
버튼 위젯을 사용하여, 인공지능 무인운반차량(AGV)이 이동해야 할 목표 방향에 해당하는 '녹색 점'을 이미지에 배치합니다.  
카메라의 이미지와 이 녹색 점의 X, Y 값을 저장합니다.  
그런 다음, 훈련 챕터에서 우리의 레이블의 X, Y 값을 예측할 신경망을 훈련시킬 것입니다. 라이브 데모에서는 예측된 X, Y 값을 사용하여 근사적인 조향 값을 계산할 것입니다.  

카메라의 라이브 비디오 피드를 살펴봅니다.  
인공지능 무인운반차량(AGV)이 따라야 할 경로를 상상해봅니다(도로에서 벗어나지 않도록 해야 할 거리를 대략적으로 추정해 봅니다).  
인공지능 무인운반차량(AGV)이 도로 밖으로 벗어나지 않고 직접 목표지점으로 향할 수 있도록 이 경로를 따라서 목표를 가능한 멀리 배치합니다.  
예를 들어, 아주 직선적인 도로인 경우 지평선에 배치할 수 있습니다. 급한 커브인 경우 인공지능 무인운반차량(AGV)에게 너무 가까이 배치하여 경계를 벗어나지 않도록 해야 할 수도 있습니다.  

## 라이브러리 가져오기   
"데이터 수집" 목적에 필요한 모든 라이브러리를 가져오는 것으로 시작해 보겠습니다. 주로 이미지를 시각화하고 라벨이 달린 이미지를 저장하기 위해 OpenCV를 사용할 것입니다. 이미지 이름 지정에는 uuid, datetime과 같은 라이브러리가 사용됩니다.  

코드를 실행합니다.

In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Robot, Camera, bgr8_to_jpeg

from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time
from SCSCtrl import TTLServo

robot = Robot()

Succeeded to open the port
Succeeded to change the baudrate


## 카메라 각도 세팅하기  
Data를 수집할 때 사용한 이미지와 맞춰서 Road Following 을 해야 합니다.  
일정한 각도로 들어오는 이미지가 필요하기 때문에, 고정된 카메라 각도를 사용합니다.   
> 주의 : 서보모터의 조립 방향에 따라 각도가 다를 수 있습니다.
본인의 서보모터 방향에 유의하여 코드를 실행합니다.  
이 코드의 목표는 Data 수집할 때 카메라 각도와 Road Following을 할 때 카메라 각도를 일치시키는 데에 있습니다.

In [2]:
TTLServo.servoAngleCtrl(5, 50, 1, 100)
print('Camera Ready!')

Camera Ready!


## AGV 제어를 위한 컨트롤러 추가하기
기존 Sample 코드에서는 조이스틱 컨트롤러를 사용하여 인공지능 무인운반차량(AGV)을 이동시키고, 이미지를 저장하는 방식을 사용하였습니다.  
하지만, 한번에 대량의 조이스틱 컨트롤러를 사용할 경우, 서로 다른 인공지능 무인운반차량(AGV)을 제어할 수 있다는 점이 발견되어, 버튼 위젯을 사용하여 인공지능 무인운반차량(AGV)을 map에서 이동시키고, 가야할 경로를 설정하고 이미지를 저장하도록 변경하였습니다.  

코드를 실행합니다.

In [3]:
# create buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)
# 레이아웃 생성 후,버튼 5개 생성

# display buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])
display(controls_box)

코드를 실행합니다.

In [4]:
def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.4)

def step_backward(change):
    robot.backward(0.4)

def step_left(change):
    robot.left(0.3)
    time.sleep(0.5)
    robot.stop()

def step_right(change):
    robot.right(0.3)
    time.sleep(0.5)
    robot.stop()

stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

이제 버튼 위젯을 눌러서 인공지능 무인운반차량(AGV)이 원활하게 이동하는 지 체크합니다.  
## 데이터 수집 경로 설정
아래 코드에서 현재 디렉토리에 ‘dataset_xy_test’ 라는 폴더를 생성하고 data를 수집합니다.  

In [5]:
DATASET_DIR = 'dataset_xy'

try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

Directories not created becasue they already exist


## 카메라 송출하기
다음은, 카메라를 사용해서, 두 개의 화면을 송출합니다.  
하나는 실시간 영상 원본을, 다른 하나는 인공지능 무인운반차량(AGV)이 나아가야 할 방향인 ‘target’의 
위치를 표시할 화면입니다.  
두 개의 x축, y축 slider를 이용해 ‘target’의 위치를 정할 수 있습니다.  

In [6]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)
display(widgets.HBox([image_widget, target_widget]))

count_widget = widgets.IntText(description='count',
 value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))
save_button = widgets.Button(description='Save', button_style='success')

display(widgets.VBox([x_slider, y_slider, count_widget, save_button]))

## 데이터 수집하기
아래 코드를 실행해서 Save 버튼을 누르면, data가 수집됩니다.  
수집할 데이터는 현재 카메라의 시야에서 인공지능 무인운반차량(AGV)이 나아가야 하는 방향을 가리키는 녹색 점의 x,y 좌표입니다.

In [7]:
def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

def save_snapshot():
    uuid = xy_uuid(x_slider.value, y_slider.value)
    image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)
    count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))
    
save_button.on_click(lambda x: save_snapshot())

데이터 수집 절차는 다음과 같습니다.  
1. Camera 의 실시간 영상을 보고, AGV가 나아가야 하는 target의 위치에 녹색 점을 둔다.
2. save 버튼을 눌러서 저장한다.
3. 수집한 데이터 파일은 dataset_xy 폴더에 저장되며, 아래와 같은 이름 형식을 지닌다.
> \"xy_xValue_yValue_UUID.jpg\"

train_model 챕터에서 model 을 학습시킬 때는 이미지 파일을 불러와서 파일 이름에서 x,y 좌표 값을 파싱해서 사용한다.

## 프로젝트 종료하기
다른 노트북에서도 사용할 수 있도록 카메라를 종료한다.

In [8]:
camera.unobserve_all()
camera.stop()
robot.stop()

## 데이터 압축해서 내보내기
data 파일을 외부 클라우드 머신에 옮기고 싶다면, 아래 명령어를 이용해서 압축한다.  
이제 다음 train_model 로 이동합니다.

In [9]:
!zip -r -q road_following_{DATASET_DIR}.zip {DATASET_DIR}